# How to save and load R objects from the workspace bucket

R users like to save intermediate work in R's native format for rapid loading.

See also [Notebooks 101 - How not to lose data output files or collaborator edits](https://broadinstitute.zendesk.com/hc/en-us/articles/360027300571-Notebooks-101-How-not-to-lose-data-output-files-or-collaborator-edits).

## Setup

First, be sure to run notebook **`R environment setup`** in this workspace.

In [1]:
library(lubridate)
library(tidyverse)
library(stringr)


Attaching package: ‘lubridate’

The following object is masked from ‘package:base’:

    date

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.0       ✔ purrr   0.3.2  
✔ tibble  2.1.1       ✔ dplyr   0.8.0.1
✔ tidyr   0.8.3       ✔ stringr 1.4.0  
✔ readr   1.3.1       ✔ forcats 0.4.0  
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ lubridate::as.difftime() masks base::as.difftime()
✖ lubridate::date()        masks base::date()
✖ dplyr::filter()          masks stats::filter()
✖ lubridate::intersect()   masks base::intersect()
✖ dplyr::lag()             masks stats::lag()
✖ lubridate::setdiff()     masks base::setdiff()
✖ lubridate::union()       masks base::union()


Get the Cloud Storage bucket associated with this workspace.

In [2]:
(WORKSPACE_BUCKET <- Sys.getenv('WORKSPACE_BUCKET'))

[1] "gs://fc-9b460d40-2985-49b5-a228-cdcf639e8c40"

In [3]:
TIMESTAMP <- strftime(now(), '%Y%m%d-%H%M%S')
(RDA_FILE <- str_glue('thousand_genomes_{TIMESTAMP}.rda'))

thousand_genomes_20190429-210705.rda

## Read some data from Cloud Storage.
Let’s retrieve the sample information for [1000 Genomes](http://www.internationalgenome.org/data "1000 Genomes").

This approach uses `gsutil cat` to transfer the contents of the CSV file since we want to load the whole thing. 

If you instead want to load a subset of columns or a subset of rows, instead retrieve the data from BigQuery table [bigquery-public-data.human_genome_variants.1000_genomes_sample_info](https://bigquery.cloud.google.com/table/bigquery-public-data:human_genome_variants.1000_genomes_sample_info).

In [4]:
df <- read_csv(pipe('gsutil cat gs://genomics-public-data/1000-genomes/other/sample_info/sample_info.csv'),
               guess_max = 5000)

Parsed with column specification:
cols(
  .default = col_character(),
  In_Low_Coverage_Pilot = col_double(),
  In_High_Coverage_Pilot = col_double(),
  In_Exon_Targetted_Pilot = col_double(),
  Has_Sequence_in_Phase1 = col_double(),
  In_Phase1_Integrated_Variant_Set = col_double(),
  Has_Phase1_chrY_SNPS = col_double(),
  Has_phase1_chrY_Deletions = col_double(),
  Has_phase1_chrMT_SNPs = col_double(),
  Total_LC_Sequence = col_double(),
  LC_Non_Duplicated_Aligned_Coverage = col_double(),
  Total_Exome_Sequence = col_double(),
  X_Targets_Covered_to_20x_or_greater = col_double(),
  VerifyBam_E_Omni_Free = col_double(),
  VerifyBam_E_Affy_Free = col_double(),
  VerifyBam_E_Omni_Chip = col_double(),
  VerifyBam_E_Affy_Chip = col_double(),
  VerifyBam_LC_Omni_Free = col_double(),
  VerifyBam_LC_Affy_Free = col_double(),
  VerifyBam_LC_Omni_Chip = col_double(),
  VerifyBam_LC_Affy_Chip = col_double()
  # ... with 11 more columns
)
See spec(...) for full column specifications.


## Save the object(s) to a local file.

In [5]:
save(df, file = RDA_FILE)

## Transfer the file to the workspace bucket

In [6]:
system(str_glue('gsutil cp {RDA_FILE} {WORKSPACE_BUCKET}/r-objects/ 2>&1'), intern = TRUE)

[1] "Copying file://thousand_genomes_20190429-210705.rda [Content-Type=application/octet-stream]..."                                                                      
[2] "/ [0 files][    0.0 B/172.9 KiB]                                                \r/ [1 files][172.9 KiB/172.9 KiB]                                                \r"
[3] "Operation completed over 1 objects/172.9 KiB.                                    "

## Now, load that object from the native format file in Cloud Storage

In [7]:
# The object exists in memory.
head(df)

Sample,Family_ID,Population,Population_Description,Gender,Relationship,Unexpected_Parent_Child,Non_Paternity,Siblings,Grandparents,⋯,In_Final_Phase_Variant_Calling,Has_Omni_Genotypes,Has_Axiom_Genotypes,Has_Affy_6_0_Genotypes,Has_Exome_LOF_Genotypes,EBV_Coverage,DNA_Source_from_Coriell,Has_Sequence_from_Blood_in_Index,Super_Population,Super_Population_Description
HG00096,HG00096,GBR,British in England and Scotland,male,NA,NA,NA,NA,NA,⋯,1,1,NA,NA,1,20.31,NA,NA,EUR,European
HG00097,HG00097,GBR,British in England and Scotland,female,NA,NA,NA,NA,NA,⋯,1,1,NA,NA,1,169.49,NA,NA,EUR,European
HG00098,HG00098,GBR,British in England and Scotland,male,NA,NA,NA,NA,NA,⋯,NA,1,NA,NA,1,NA,NA,NA,EUR,European
HG00099,HG00099,GBR,British in England and Scotland,female,NA,NA,NA,NA,NA,⋯,1,1,NA,NA,1,23.04,NA,NA,EUR,European
HG00100,HG00100,GBR,British in England and Scotland,female,NA,NA,NA,NA,NA,⋯,1,1,NA,NA,1,116.22,NA,NA,EUR,European
HG00101,HG00101,GBR,British in England and Scotland,male,NA,NA,NA,NA,NA,⋯,1,1,NA,NA,1,82.00,NA,NA,EUR,European


In [8]:
# Go ahead and delete it.
rm(df)

In [9]:
# Okay, its gone.
head(df)

                                              
1 function (x, df1, df2, ncp, log = FALSE)    
2 {                                           
3     if (missing(ncp))                       
4         .Call(C_df, x, df1, df2, log)       
5     else .Call(C_dnf, x, df1, df2, ncp, log)
6 }                                           

In [10]:
load(pipe(str_glue('gsutil cat {WORKSPACE_BUCKET}/r-objects/{RDA_FILE}')))

In [11]:
# The object exists in memory again!
head(df)

Sample,Family_ID,Population,Population_Description,Gender,Relationship,Unexpected_Parent_Child,Non_Paternity,Siblings,Grandparents,⋯,In_Final_Phase_Variant_Calling,Has_Omni_Genotypes,Has_Axiom_Genotypes,Has_Affy_6_0_Genotypes,Has_Exome_LOF_Genotypes,EBV_Coverage,DNA_Source_from_Coriell,Has_Sequence_from_Blood_in_Index,Super_Population,Super_Population_Description
HG00096,HG00096,GBR,British in England and Scotland,male,NA,NA,NA,NA,NA,⋯,1,1,NA,NA,1,20.31,NA,NA,EUR,European
HG00097,HG00097,GBR,British in England and Scotland,female,NA,NA,NA,NA,NA,⋯,1,1,NA,NA,1,169.49,NA,NA,EUR,European
HG00098,HG00098,GBR,British in England and Scotland,male,NA,NA,NA,NA,NA,⋯,NA,1,NA,NA,1,NA,NA,NA,EUR,European
HG00099,HG00099,GBR,British in England and Scotland,female,NA,NA,NA,NA,NA,⋯,1,1,NA,NA,1,23.04,NA,NA,EUR,European
HG00100,HG00100,GBR,British in England and Scotland,female,NA,NA,NA,NA,NA,⋯,1,1,NA,NA,1,116.22,NA,NA,EUR,European
HG00101,HG00101,GBR,British in England and Scotland,male,NA,NA,NA,NA,NA,⋯,1,1,NA,NA,1,82.00,NA,NA,EUR,European


# Provenance

In [12]:
devtools::session_info()

─ Session info ───────────────────────────────────────────────────────────────
 setting  value                       
 version  R version 3.5.2 (2018-12-20)
 os       Debian GNU/Linux 9 (stretch)
 system   x86_64, linux-gnu           
 ui       X11                         
 language (EN)                        
 collate  en_US.UTF-8                 
 ctype    en_US.UTF-8                 
 tz       Etc/UTC                     
 date     2019-04-29                  

─ Packages ───────────────────────────────────────────────────────────────────
 package     * version     date       lib source                            
 assertthat    0.2.1       2019-03-21 [2] CRAN (R 3.5.2)                    
 backports     1.1.3       2018-12-14 [2] CRAN (R 3.5.2)                    
 base64enc     0.1-3       2015-07-28 [2] CRAN (R 3.5.2)                    
 broom         0.5.1       2018-12-05 [2] CRAN (R 3.5.2)                    
 callr         3.2.0       2019-03-15 [2] CRAN (R 3.5.2)          

Copyright 2018 The Broad Institute, Inc., Verily Life Sciences, LLC All rights reserved.

This software may be modified and distributed under the terms of the BSD license. See the LICENSE file for details.